<a href="https://colab.research.google.com/github/hrbolek/learning/blob/master/operanalyst/teoriegrafu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teorie grafů

## Matematický popis

$$G=(V,E)$$
$V$ je množina vrcholů, $E\subseteq\{\{x,y\}|x,y \in V, x \ne y\}$ je množina hran.

Rozlišujeme neorientované a orientované grafy.

## Grafické vyjádření